In [10]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio.set_index('Ticker', inplace=True)

portfolio = portfolio[['% Allocation','CVaR']]

portfolio['CVaR'] = portfolio['CVaR'].astype(float)
portfolio['% Allocation'] = portfolio['% Allocation'].astype(float)

portfolio = portfolio[['CVaR']]

portfolio.reset_index(inplace=True)
portfolio.rename(columns={'index': 'Ticker'}, inplace=True)

portfolio

,Ticker,CVaR
0,CEG,0.077
1,AAPL,0.071
2,RTX,0.068
3,PANW,0.048
4,NOW,0.038
5,AMZN,0.050
6,CVX,0.062
7,JPM,0.065
8,MSFT,0.086
9,LOW,0.056


In [12]:
portfolio_value = 20639

end = pd.Timestamp(datetime.date.today())
start = end - relativedelta(years=1)
#start = '2023-11-16'

symbols = portfolio['Ticker'].tolist()

#stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data = openbb.stocks.ca.hist(symbols,start, end)

#stock_data.dropna(how="any", axis=1, inplace=True)

stock_data.reset_index(inplace=True)
#stock_data.rename(columns={'index': 'Ticker'}, inplace=True)

stock_data



,Date,CEG,AAPL,RTX,PANW,NOW,AMZN,CVX,JPM,MSFT,LOW,FTNT,ULTA,CMI,NVDA,GEHC
0,2022-12-16,87.138611,133.762329,95.824394,151.000000,394.369995,87.860001,162.341461,125.547615,242.539429,201.919098,51.919998,451.119995,229.017853,165.636505,56.671654
1,2022-12-19,86.674263,131.634232,95.044861,150.520004,382.299988,84.919998,163.457581,126.295326,238.336685,199.009918,49.980000,440.510010,230.563446,162.467911,56.431946
2,2022-12-20,86.506317,131.564636,95.668495,149.279999,386.470001,85.190002,166.142120,126.907120,239.674820,196.747208,50.470001,439.200012,230.825912,160.778656,57.141087
3,2022-12-21,88.571175,134.697098,97.208069,147.419998,391.910004,86.769997,168.095367,128.334549,242.281693,198.285065,50.009998,452.290009,235.452911,164.936798,56.891388
4,2022-12-22,86.279091,131.495010,95.990051,143.149994,379.940002,83.790001,165.574417,126.877975,236.096558,195.072205,49.150002,457.049988,233.955948,153.321960,57.530617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2023-12-11,116.150002,193.179993,81.889999,299.880005,705.119995,145.889999,144.350006,159.100006,371.299988,206.339996,53.639999,482.640015,235.320007,466.269989,71.129997
247,2023-12-12,114.489998,194.710007,81.040001,305.579987,716.479980,147.479996,142.500000,160.520004,374.380005,208.699997,55.040001,493.029999,234.669998,476.570007,72.050003
248,2023-12-13,116.430000,197.960007,83.220001,313.859985,715.640015,148.839996,144.610001,161.059998,374.369995,215.449997,55.349998,493.119995,239.350006,480.880005,74.180000
249,2023-12-14,115.599998,198.110001,81.860001,300.440002,680.390015,147.419998,149.929993,163.990005,365.929993,225.539993,57.049999,494.630005,248.270004,483.500000,75.919998


In [13]:
results = pd.DataFrame(columns=['Date', 'Cumulative_Portfolio_Value'])

# Assuming that your price_data DataFrame has a 'Date' column, include it in the results DataFrame.
results['Date'] = stock_data['Date']

# Initialize the first row with starting values
results.at[0, 'Cumulative_Portfolio_Value'] = portfolio_value
for ticker in portfolio['Ticker']:
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    results.at[0, f'{ticker}_Contribution'] = portfolio_value * allocation

# Iterate through stock tickers and calculate portfolio value and contributions
for ticker in portfolio['Ticker']:
    returns_column = f'{ticker}_Return'
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    for i in range(1, len(results)):
        results.at[i, f'{ticker}_Return'] = stock_data.at[i, ticker] / stock_data.at[i - 1, ticker] - 1
        results.at[i, f'{ticker}_Contribution'] = results.at[i - 1, f'{ticker}_Contribution'] * (1 + results.at[i, f'{ticker}_Return'])

# Calculate daily portfolio value and cumulative portfolio value
for i in range(1, len(results)):
    cumulative_portfolio_value = portfolio_value
    for ticker in portfolio['Ticker']:
        contribution_column = f'{ticker}_Contribution'
        cumulative_portfolio_value += results.at[i, contribution_column]
    results.at[i, 'Cumulative_Portfolio_Value'] = cumulative_portfolio_value


results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,AMZN_Return,CVX_Return,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return
0,2022-12-16,20639,1589.203000,1465.369000,1403.452000,990.672000,784.282000,1031.950000,1279.618000,1341.535000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-19,41056.422895,1580.734389,1442.055649,1392.034890,987.522872,760.278426,997.418524,1288.415546,1349.524644,...,-0.033462,0.006875,0.005956,-0.017328,-0.014408,-0.037365,-0.023519,0.006749,-0.019130,-0.004230
2,2022-12-20,41103.073689,1577.671452,1441.293230,1401.168690,979.387516,768.571314,1000.589829,1309.575793,1356.061944,...,0.003180,0.016423,0.004844,0.005614,-0.011370,0.009804,-0.002974,0.001138,-0.010397,0.012566
3,2022-12-21,41382.115104,1615.329589,1475.609410,1423.717413,967.184533,779.389825,1019.147478,1324.971799,1371.314692,...,0.018547,0.011756,0.011248,0.010877,0.007816,-0.009114,0.029804,0.020045,0.025863,-0.004370
4,2022-12-22,41012.831277,1573.527381,1440.530478,1405.878227,939.170137,755.585132,984.146265,1305.101007,1355.750523,...,-0.034344,-0.014997,-0.011350,-0.025529,-0.016203,-0.017196,0.010524,-0.006358,-0.070420,0.011236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2023-12-11,49313.156177,2118.302428,2116.290697,1199.367701,1967.435260,1402.269257,1713.534985,1137.804630,1700.057995,...,-0.010379,0.000277,0.003659,-0.007829,-0.007169,0.028374,0.000996,0.020557,-0.018503,0.038091
247,2023-12-12,49597.453086,2088.027871,2133.052032,1186.918553,2004.831367,1424.860813,1732.210113,1123.222396,1715.231339,...,0.010899,-0.012816,0.008925,0.008295,0.011437,0.026100,0.021527,-0.002762,0.022090,0.012934
248,2023-12-13,49982.348938,2123.408946,2168.655847,1218.847018,2059.154301,1423.190377,1748.183851,1139.853975,1721.001420,...,0.009222,0.014807,0.003364,-0.000027,0.032343,0.005632,0.000183,0.019943,0.009044,0.029563
249,2023-12-14,50019.848038,2108.271667,2170.299033,1198.928337,1971.109259,1353.088846,1731.505418,1181.787548,1752.309926,...,-0.009540,0.036789,0.018192,-0.022545,0.046832,0.030714,0.003062,0.037268,0.005448,0.023456


In [14]:
# Step 1: Select columns containing the word 'Contribution'
contribution_columns = [col for col in results.columns if 'Contribution' in col]

# Step 2: Create a new column that sums each row of 'Contribution' columns
results['Total_Contribution'] = results[contribution_columns].sum(axis=1)

results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,CVX_Return,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return,Total_Contribution
0,2022-12-16,20639,1589.203000,1465.369000,1403.452000,990.672000,784.282000,1031.950000,1279.618000,1341.535000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20659.639000
1,2022-12-19,41056.422895,1580.734389,1442.055649,1392.034890,987.522872,760.278426,997.418524,1288.415546,1349.524644,...,0.006875,0.005956,-0.017328,-0.014408,-0.037365,-0.023519,0.006749,-0.019130,-0.004230,20417.422895
2,2022-12-20,41103.073689,1577.671452,1441.293230,1401.168690,979.387516,768.571314,1000.589829,1309.575793,1356.061944,...,0.016423,0.004844,0.005614,-0.011370,0.009804,-0.002974,0.001138,-0.010397,0.012566,20464.073689
3,2022-12-21,41382.115104,1615.329589,1475.609410,1423.717413,967.184533,779.389825,1019.147478,1324.971799,1371.314692,...,0.011756,0.011248,0.010877,0.007816,-0.009114,0.029804,0.020045,0.025863,-0.004370,20743.115104
4,2022-12-22,41012.831277,1573.527381,1440.530478,1405.878227,939.170137,755.585132,984.146265,1305.101007,1355.750523,...,-0.014997,-0.011350,-0.025529,-0.016203,-0.017196,0.010524,-0.006358,-0.070420,0.011236,20373.831277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2023-12-11,49313.156177,2118.302428,2116.290697,1199.367701,1967.435260,1402.269257,1713.534985,1137.804630,1700.057995,...,0.000277,0.003659,-0.007829,-0.007169,0.028374,0.000996,0.020557,-0.018503,0.038091,28674.156177
247,2023-12-12,49597.453086,2088.027871,2133.052032,1186.918553,2004.831367,1424.860813,1732.210113,1123.222396,1715.231339,...,-0.012816,0.008925,0.008295,0.011437,0.026100,0.021527,-0.002762,0.022090,0.012934,28958.453086
248,2023-12-13,49982.348938,2123.408946,2168.655847,1218.847018,2059.154301,1423.190377,1748.183851,1139.853975,1721.001420,...,0.014807,0.003364,-0.000027,0.032343,0.005632,0.000183,0.019943,0.009044,0.029563,29343.348938
249,2023-12-14,50019.848038,2108.271667,2170.299033,1198.928337,1971.109259,1353.088846,1731.505418,1181.787548,1752.309926,...,0.036789,0.018192,-0.022545,0.046832,0.030714,0.003062,0.037268,0.005448,0.023456,29380.848038


In [15]:
# Calculate the percentage gain/loss relative to the first row
results['Percentage_Gain_Loss'] = ((results['Total_Contribution'] - results['Total_Contribution'].iloc[0]) / results['Total_Contribution'].iloc[0])

results 

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return,Total_Contribution,Percentage_Gain_Loss
0,2022-12-16,20639,1589.203000,1465.369000,1403.452000,990.672000,784.282000,1031.950000,1279.618000,1341.535000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20659.639000,0.000000
1,2022-12-19,41056.422895,1580.734389,1442.055649,1392.034890,987.522872,760.278426,997.418524,1288.415546,1349.524644,...,0.005956,-0.017328,-0.014408,-0.037365,-0.023519,0.006749,-0.019130,-0.004230,20417.422895,-0.011724
2,2022-12-20,41103.073689,1577.671452,1441.293230,1401.168690,979.387516,768.571314,1000.589829,1309.575793,1356.061944,...,0.004844,0.005614,-0.011370,0.009804,-0.002974,0.001138,-0.010397,0.012566,20464.073689,-0.009466
3,2022-12-21,41382.115104,1615.329589,1475.609410,1423.717413,967.184533,779.389825,1019.147478,1324.971799,1371.314692,...,0.011248,0.010877,0.007816,-0.009114,0.029804,0.020045,0.025863,-0.004370,20743.115104,0.004041
4,2022-12-22,41012.831277,1573.527381,1440.530478,1405.878227,939.170137,755.585132,984.146265,1305.101007,1355.750523,...,-0.011350,-0.025529,-0.016203,-0.017196,0.010524,-0.006358,-0.070420,0.011236,20373.831277,-0.013834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2023-12-11,49313.156177,2118.302428,2116.290697,1199.367701,1967.435260,1402.269257,1713.534985,1137.804630,1700.057995,...,0.003659,-0.007829,-0.007169,0.028374,0.000996,0.020557,-0.018503,0.038091,28674.156177,0.387931
247,2023-12-12,49597.453086,2088.027871,2133.052032,1186.918553,2004.831367,1424.860813,1732.210113,1123.222396,1715.231339,...,0.008925,0.008295,0.011437,0.026100,0.021527,-0.002762,0.022090,0.012934,28958.453086,0.401692
248,2023-12-13,49982.348938,2123.408946,2168.655847,1218.847018,2059.154301,1423.190377,1748.183851,1139.853975,1721.001420,...,0.003364,-0.000027,0.032343,0.005632,0.000183,0.019943,0.009044,0.029563,29343.348938,0.420322
249,2023-12-14,50019.848038,2108.271667,2170.299033,1198.928337,1971.109259,1353.088846,1731.505418,1181.787548,1752.309926,...,0.018192,-0.022545,0.046832,0.030714,0.003062,0.037268,0.005448,0.023456,29380.848038,0.422138


In [16]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(4)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,results)